In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [20]:
headers = {
  'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
}

Wyłuskanie interesujących z punktu analizy kolumn

In [21]:
def get_number_of_pages(league,year):
    page = f"https://www.transfermarkt.pl/championship/toptransfers/wettbewerb/{league}/saison_id/{year}/land_id/alle/ausrichtung//spielerposition_id/alle/altersklasse//w_s//zuab/zu/art//plus/1/galerie/0/page/1"
    treeSoup = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(treeSoup.content, 'html.parser')
    size = pageSoup.find_all("li", {"class": "tm-pagination__list-item tm-pagination__list-item--icon-last-page"})
    size = str(size).split("(strona ")[1].split(")")[0]
    return(int(size))    


Funkcja sprawadzajaca czy tag jest zgodny z wymaganiami

In [22]:
def check_club(tag):
    is_valid = False
    if tag.has_attr('class') and 'tiny_wappen' in tag.attrs['class']:
        is_valid = True
    elif tag.has_attr('title'):
        is_valid = tag.attrs['title'] == '\xa0'

    return is_valid


In [23]:
def scrape_data(leagues : list(), years : list()):
    club = [] #list of clubs which loan in
    players = [] #name of player
    values = [] #value in moment of transfer
    for league in leagues:
        print(league)
        for year in years:
            print(year)
            size = get_number_of_pages(league,year)
            for i in range(1,size+1):
                page = f"https://www.transfermarkt.pl/championship/toptransfers/wettbewerb/{league}/saison_id/{year}/land_id/alle/ausrichtung//spielerposition_id/alle/altersklasse//w_s//zuab/zu/art//plus/1/galerie/0/page/{i}"
                treeSoup = requests.get(page, headers=headers)
                pageSoup = BeautifulSoup(treeSoup.content, 'html.parser')
                players.extend(pageSoup.find_all("img", {"class" : "bilderrahmen-fixed lazy lazy"}))
                values.extend(pageSoup.find_all("td", {"class": ["rechts", "rechts bg_gruen_20"]}))
                club.extend(pageSoup.find_all(check_club))
                print(i)
    print(f"Graczy: {len(players)}, Klubów: {len(club)}, Wartości: {len(values)}")
    return (players,club,values)

Wyłuskanie wartości z wierszy

In [24]:
def get_loan_data(players, club, values):
    players_list = []
    club_list = []
    values_list = []
    price_list = []
    for line in players:
        players_list.append(str(line).split('" class')[0].split('<img alt="')[1])
    for i in range(1,len(club)+1,2):
        club_list.append(str(club[i]).split('img alt="')[1].split('" class')[0])
    for i in range(0,len(values),2):
        values_list.append(str(values[i]).split("</td>")[0].split('">')[1])
        price_list.append(str(values[i+1]).split("</a>")[0].split('">')[2])
    len(values_list)
    df = pd.DataFrame({"Player" : players_list, "Loaning club" : club_list, "Player value in moment of transfer": values_list, "Transfer cost": price_list})
    loan_df = df.loc[df['Transfer cost'] == "Wypożyczenie"]
    return(loan_df)

Tabela danych do analizy

In [25]:
data = scrape_data(["GB2"],["2022","2021","2020"])
loan_df = get_loan_data(data[0],data[1],data[2])
loan_df

GB2
2022
1
2
3
4
5
6
7
8
9
10
11
12
13
14
2021
1
2
3
4
5
6
7
8
9
10
11
12
13
2020
1
2
3
4
5
6
7
8
9
10
11
12
13
Graczy: 963, Klubów: 1926, Wartości: 1926


,Player,Loaning club,Player value in moment of transfer,Transfer cost
0,Amad Diallo,AFC Sunderland,"15,00 mln €",Wypożyczenie
3,Marquinhos,Norwich City,"10,00 mln €",Wypożyczenie
4,Alex Mowatt,FC Middlesbrough,"10,00 mln €",Wypożyczenie
6,Cameron Archer,FC Middlesbrough,"8,00 mln €",Wypożyczenie
7,Tyler Roberts,Queens Park Rangers,"8,00 mln €",Wypożyczenie
...,...,...,...,...
909,Josh Knight,Wycombe Wanderers,-,Wypożyczenie
923,Sam Nombe,Luton Town,-,Wypożyczenie
939,Ryan Giles,Coventry City,-,Wypożyczenie
941,Ben Sheaf,Coventry City,-,Wypożyczenie


In [26]:
"""page = "https://www.transfermarkt.pl/manchester-united/startseite/verein/985"
treeSoup = requests.get(page, headers=headers)
pageSoup = BeautifulSoup(treeSoup.content, 'html.parser')
print(pageSoup)
gracze = []
gracze = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
gracze"""

'page = "https://www.transfermarkt.pl/manchester-united/startseite/verein/985"\ntreeSoup = requests.get(page, headers=headers)\npageSoup = BeautifulSoup(treeSoup.content, \'html.parser\')\nprint(pageSoup)\ngracze = []\ngracze = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})\ngracze'